In [9]:
import osmnx as ox
import plotly.graph_objects as go
from shapely.geometry import LineString
import numpy as np

In [10]:
# Configure OSMnx to use a valid user agent
ox.settings.log_console = True
ox.settings.use_cache = True  # Cache responses to avoid overloading servers

In [11]:
# Fetch street network (simplify to clean geometries)
place = "Madrid, Spain"
try:
    graph = ox.graph_from_place(place, network_type="drive", simplify=True)
except Exception as e:
    print(f"Failed to fetch graph: {e}")
    exit()

In [18]:
# Define routes with multiple stops (lists of points)
routes = [
    # Route 1: Puerta del Sol → Plaza Mayor → Royal Palace
    [
        (40.4167, -3.70325),  # Puerta del Sol
        (40.4155, -3.7074),   # Plaza Mayor
        (40.4181, -3.7142)    # Royal Palace
    ],
    # Route 2: Santiago Bernabéu → Cibeles → Retiro Park
    [
        (40.4530, -3.6883),   # Santiago Bernabéu
        (40.4192, -3.6928),   # Cibeles
        (40.4142, -3.6831)    # Retiro Park
    ]
]

In [19]:
# Colors and marker symbols
colors = ['blue', 'green']
marker_symbols = ['triangle-up', 'circle', 'square']  # Start, Stop, End

In [20]:
# Initialize Plotly figure
fig = go.Figure()

In [33]:
for route_idx, route_points in enumerate(routes):
    route_color = colors[route_idx]
    route_coords = []  # To store all coordinates for the route
    
    # Process each segment (e.g., point1 → point2 → point3)
    for i in range(len(route_points) - 1):
        start_point = route_points[i]
        end_point = route_points[i + 1]
        
        # Find nearest nodes (ensure lon, lat order for OSMnx)
        start_node = ox.nearest_nodes(graph, X=start_point[1], Y=start_point[0])
        end_node = ox.nearest_nodes(graph, X=end_point[1], Y=end_point[0])
        
        # Compute shortest path for this segment
        try:
            segment_path = ox.shortest_path(graph, start_node, end_node, weight="length")
            if not segment_path:
                print(f"No path found for segment {i+1} in Route {route_idx+1}")
                continue
        except ox.exceptions.NoPath:
            print(f"No path for segment {i+1} in Route {route_idx+1}")
            continue
        
        # Extract edges for this segment
        segment_edges = []
        for u, v in zip(segment_path[:-1], segment_path[1:]):
            edge_data = graph.get_edge_data(u, v)[0]
            segment_edges.append((u, v, edge_data))
        
        # Extract coordinates for this segment
        for u, v, edge in segment_edges:
            if 'geometry' in edge:
                line = edge['geometry']
                coords = list(line.coords)
            else:
                # Use node coordinates if geometry is missing
                u_lon, u_lat = graph.nodes[u]['x'], graph.nodes[u]['y']
                v_lon, v_lat = graph.nodes[v]['x'], graph.nodes[v]['y']
                line = LineString([(u_lon, u_lat), (v_lon, v_lat)])
                coords = list(line.coords)
            
            # Convert (lon, lat) → (lat, lon) and clean duplicates
            previous_point = None
            for lon, lat in coords:
                current_point = (lat, lon)
                if current_point != previous_point:
                    route_coords.append(current_point)
                    previous_point = current_point
    
    # Add the entire route as a continuous line
    if route_coords:
        lons = [lon for lat, lon in route_coords]
        lats = [lat for lat, lon in route_coords]
        fig.add_trace(go.Scattermapbox(
            mode="lines",
            lon=lons,
            lat=lats,
            line=dict(width=3, color=route_color),
            name=f"Route {route_idx+1}"
        ))
    
    # Add markers for each stop (start, intermediate, end)
    for point_idx, (lat, lon) in enumerate(route_points):
        fig.add_trace(go.Scattermapbox(
            mode="markers+text",
            lon=[lon],
            lat=[lat],
            marker=dict(size=14, color="red", symbol="circle"),
            text="Pa que sepa",
            textposition="bottom right",
            showlegend=False
        ))


In [35]:
# Configure map layout
all_lats = [point[0] for route in routes for point in route]
all_lons = [point[1] for route in routes for point in route]
fig.update_layout(
    mapbox=dict(
        style="open-street-map",
        zoom=13,
        center=dict(lat=np.mean(all_lats), lon=np.mean(all_lons)),
    ),
    margin=dict(l=0, r=0, t=30, b=0),
    height=800,
    width=800,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()